# General Usage

In [1]:
from pprint import pprint
import tdameritrade as td

## Authentication



In [2]:
client_id = ""
refresh_token = ""

In [4]:
# Authorize TD Client
td_client = td.TDClient(client_id=client_id, refresh_token=refresh_token)

## API Examples

In [5]:
# shows private account info
# pprint(td_client.accounts())

### Getting Symbol Data

In [6]:
pprint(td_client.search('aapl'))

{'AAPL': {'assetType': 'EQUITY',
          'cusip': '037833100',
          'description': 'Apple Inc. - Common Stock',
          'exchange': 'NASDAQ',
          'symbol': 'AAPL'}}


In [7]:
pprint(td_client.quote('aapl'))

{'AAPL': {'52WkHigh': 356.56,
          '52WkLow': 192.58,
          'askId': 'K',
          'askPrice': 347.2,
          'askSize': 100,
          'assetMainType': 'EQUITY',
          'assetType': 'EQUITY',
          'bidId': 'K',
          'bidPrice': 347.07,
          'bidSize': 600,
          'bidTick': ' ',
          'closePrice': 351.73,
          'cusip': '037833100',
          'delayed': False,
          'description': 'Apple Inc. - Common Stock',
          'digits': 4,
          'divAmount': 3.28,
          'divDate': '2020-05-08 00:00:00.000',
          'divYield': 0.93,
          'exchange': 'q',
          'exchangeName': 'NASD',
          'highPrice': 356.56,
          'lastId': 'D',
          'lastPrice': 347.135,
          'lastSize': 0,
          'lowPrice': 345.15,
          'marginable': True,
          'mark': 347.2,
          'markChangeInDouble': -4.53,
          'markPercentChangeInDouble': -1.2879,
          'nAV': 0.0,
          'netChange': -4.595,
          'ne

In [8]:
pprint(td_client.instrument('037833100'))

[{'assetType': 'EQUITY',
  'cusip': '037833100',
  'description': 'Apple Inc. - Common Stock',
  'exchange': 'NASDAQ',
  'symbol': 'AAPL'}]


In [9]:
import pandas as pd
pd.DataFrame(td_client.history('AAPL')['candles']).head()

,close,datetime,high,low,open,volume
0,323.80,1591354800000,323.95,323.61,323.61,2395
1,323.75,1591354860000,323.81,323.75,323.80,1338
2,323.78,1591354920000,323.80,323.76,323.80,423
3,323.65,1591355040000,323.70,323.65,323.70,210
4,323.77,1591355160000,323.77,323.68,323.70,1159


### Getting Option Data

In [10]:
options = td_client.options('AAPL')

In [11]:
expiry_dates = list(options['putExpDateMap'].keys())
expiry = expiry_dates[0]
print(f"Expiry Date: {expiry}")

strike_prices = list(options['putExpDateMap'][expiry].keys())
strike = strike_prices[0]
print(f"Strike Price: {strike}")

pprint(options['putExpDateMap'][expiry][strike])

Expiry Date: 2020-06-19:0
Strike Price: 75.0
[{'ask': 0.01,
  'askSize': 865,
  'bid': 0.0,
  'bidAskSize': '0X865',
  'bidSize': 0,
  'closePrice': 0.0,
  'daysToExpiration': 0,
  'deliverableNote': '',
  'delta': 0.0,
  'description': 'AAPL Jun 19 2020 75.0 Put',
  'exchangeName': 'OPR',
  'expirationDate': 1592596800000,
  'expirationType': 'R',
  'gamma': 0.0,
  'highPrice': 0.0,
  'inTheMoney': False,
  'isIndexOption': None,
  'last': 0.01,
  'lastSize': 0,
  'lastTradingDay': 1592611200000,
  'lowPrice': 0.0,
  'mark': 0.01,
  'markChange': 0.0,
  'markPercentChange': 4900.0,
  'mini': False,
  'multiplier': 100.0,
  'netChange': 0.01,
  'nonStandard': False,
  'openInterest': 7941,
  'openPrice': 0.0,
  'optionDeliverablesList': None,
  'percentChange': 9900.0,
  'putCall': 'PUT',
  'quoteTimeInLong': 1592574187630,
  'rho': 0.0,
  'settlementType': ' ',
  'strikePrice': 75.0,
  'symbol': 'AAPL_061920P75',
  'theoreticalOptionValue': 0.005,
  'theoreticalVolatility': 29.0,
  't

## Placing an Order

### Creating order body for Place Order API

In [12]:
from tdameritrade.orders.order_builder import build_buy_market_stock_order

# Prints function docstring
# build_buy_market_stock_order?

In [13]:
symbol = "HUSA"  # Cheap stock for example
quantity = 1
order = build_buy_market_stock_order(symbol, quantity)

In [14]:
pprint(order.asdict())

{'duration': 'DAY',
 'orderLegCollection': [{'instruction': 'BUY',
                         'instrument': {'assetType': 'EQUITY',
                                        'symbol': 'HUSA'},
                         'quantity': 1}],
 'orderStrategyType': 'SINGLE',
 'orderType': 'MARKET',
 'session': 'NORMAL'}


### Submitting Order

In [15]:
# Without this, it fails with a 415.
# This may screw up other API calls, so it may have to be reset if you want to try other ones.
td_client.session.headers = {
    "Content-Type": "application/json"
}

In [16]:
# This will fail to return because the place order API does not return a JSONserializable response on success
# BUT the order will still go through.  Checked on the site.  You can also check your active orders below.

# try:
#     response = td_client.placeOrder(account_id, order.json())
#     pprint(response)
# except json.decoder.JSONDecodeError as err:
#     print(err)


### Getting out Orders

In [17]:
# Load account info to get account ID for get orders
td_client.accounts()

account_id = td_client.accountIds[0]

In [18]:
# Print out orders
my_orders = td_client.orders(accountId=account_id)
pprint(my_orders)

[]


### Deleting Orders

In [19]:
# This deletes all WORKING orders

# for order in my_orders:
#     if order["status"] == Status.WORKING and order["cancelable"]:
#         response = td_client.cancelOrder(account_id, order["orderId"])
#         pprint(response)